In [2]:
from tweepy.streaming import StreamListener
from tweepy.auth import OAuthHandler
from tweepy import Stream

from datetime import datetime
import json


import pandas as pd
from pymongo import MongoClient

In [4]:
#pyenv
import os
from dotenv import load_dotenv
load_dotenv()
CONSUMER_KEY = os.environ.get('CONSUMER_KEY')
CONSUMER_SECRET = os.environ.get('CONSUMER_SECRET')
ACCESS_TOKEN = os.environ.get('ACCESS_TOKEN')
ACCESS_TOKEN_SECRET = os.environ.get('ACCESS_TOKEN_SECRET')

In [6]:
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
#api = tweepy.API(auth)
#api.update_status('tweepy + oauth!')

In [7]:
class MyListener(StreamListener):
    def on_data(self,data):
        print("ayn")
        print(data)
        tweet = json.loads(data)
        created_at = tweet["created_at"]
        id_str = tweet[ "id_str"]
        text = tweet[ "text"]
        obj = {"created_at":created_at, "id_str":id_str, "text":text}
        tweetind = col.insert_one(obj).inserted_id
        print(obj)
        return True
    
    def on_error(self, status):
        print(status)
        return False

In [8]:
mylistener = MyListener()
mylistener

In [9]:
mystream = Stream(auth, listener = mylistener)
mystream

In [10]:
client = MongoClient('localhost', 27017)
db = client.twitterdb
col = db.tweets
keywords = ['Hello']

In [11]:
print('inicia stream')
#mystream.filter(track=keywords)
mystream.filter(track=['bieber'])
print('fecha stream')

inicia stream
ayn
{"created_at":"Thu May 02 21:43:18 +0000 2019","id":1124066860381683712,"id_str":"1124066860381683712","text":"RT @wifeyhails: justin bieber ft. ed sheeran\njustin bieber ft. chris brown\njustin bieber ft. billie eilish\n\nwe\u2018re finally getting fed this\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1087463260964700160,"id_str":"1087463260964700160","name":"Lariss\u00aa","screen_name":"larissinhabibzz","location":"Infelizmente no Brazil","url":null,"description":"A","translator_type":"none","protected":false,"verified":false,"followers_count":73,"friends_count":400,"listed_count":0,"favourites_count":9434,"statuses_count":2685,"created_at":"Mon Jan 21 21:33:40 +0000 2019","utc_offset":null,"time_zone":n

KeyboardInterrupt: 

In [ ]:
class LimitedListener(StreamListener):

    def __init__(self):
        super().__init__()
        self.counter = 0
        self.limit = 10

    def on_status(self, status):
        try:
            userid = status.user.id_str
            geo = str(status.coordinates)
            if geo != "None":
                print(userid + ',' + geo)
            else:
                print("No coordinates")
            self.counter += 1
            if self.counter < self.limit:
                return True
            else:
                twitterStream.disconnect()
        except BaseException as e:
            print('failed on_status,',str(e))
            time.sleep(5)

In [ ]:
limited_listener = LimitedListener()
limited_stream = Stream(auth, listener = limited_listener)
print('inicia limited stream')
limited_stream.filter(track=keywords)
print('fecha limited stream')

In [ ]:
"""
client = MongoClient('localhost', 27017)
db_test = client.testdb
collection_test = db_test.col_test
obj_test = {"putetinho": "ayn ayn"}
collection_test.insert_one(obj_test)
"""

In [ ]:
col.find_one()

In [ ]:
dataset = [{"created_at":item["created_at"],
           "text":item["text"],
           } for item in col.find()]

In [ ]:
df = pd.DataFrame(dataset)

In [ ]:
df.head()

In [ ]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df.text)

In [ ]:
word_count = pd.DataFrame(cv.get_feature_names(), columns=["word"])
word_count["count"] = count_matrix.sum(axis=0).tolist()[0]
word_count = word_count.sort_values("count", ascending=False).reset_index(drop=True)
word_count[:50]